# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.

%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
import dask.dataframe as dd

/var/folders/_4/g68nw_rd0rq7p11rzh63s98m0000gn/T/ipykernel_35644/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
parquet_files


['../../05_src/data/prices/CTAS/CTAS_2008.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2018.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2011.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2001.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2000.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2010.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2019.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2009.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2012.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2002.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2024.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2003.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2013.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2016.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2006.parquet/part.0.parqu

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
# Write your code below.

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1),
                       Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))



/opt/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/var/folders/_4/g68nw_rd0rq7p11rzh63s98m0000gn/T/ipykernel_35644/140552518.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [7]:
# Write your code below.

import pandas as pd
import numpy as np

df = dd_feat.compute()
df = (df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_avg_return = x['returns'].rolling(10).mean())
))
df


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,,
HUM,2003-01-02,10.100000,10.350000,10.030000,10.31,9.205754,856400,Health Care,Managed Health Care,2003,NaN,NaN,NaN,0.320001,NaN
HUM,2003-01-03,10.320000,10.480000,10.310000,10.40,9.286116,795300,Health Care,Managed Health Care,2003,10.310000,9.205754,0.008729,0.169999,NaN
HUM,2003-01-06,10.480000,10.500000,10.260000,10.32,9.214683,962000,Health Care,Managed Health Care,2003,10.400000,9.286116,-0.007692,0.240000,NaN
HUM,2003-01-07,10.260000,10.400000,10.250000,10.31,9.205754,980000,Health Care,Managed Health Care,2003,10.320000,9.214683,-0.000969,0.150000,NaN
HUM,2003-01-08,10.320000,10.440000,10.250000,10.40,9.286116,1071600,Health Care,Managed Health Care,2003,10.310000,9.205754,0.008729,0.190000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2014-12-24,22.790001,23.040001,22.780001,22.83,17.948908,1338700,Information Technology,Communications Equipment,2014,22.809999,17.933178,0.000877,0.260000,0.006406
JNPR,2014-12-26,22.870001,22.980000,22.719999,22.73,17.870289,1982600,Information Technology,Communications Equipment,2014,22.830000,17.948908,-0.004380,0.260000,0.006248
JNPR,2014-12-29,22.730000,22.850000,22.580000,22.59,17.760216,2569700,Information Technology,Communications Equipment,2014,22.730000,17.870289,-0.006160,0.270000,0.006895


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?


No, it was not strictly necessary to convert to Pandas to calculate the moving average return. Dask does support .rolling() operations, including .rolling().mean(), which allows for calculating moving averages directly in Dask without converting to Pandas. However, depending on the size of the dataset and the specifics of this assignment environment, converting to Pandas might simplify the process and offer some performance advantages for smaller datasets.




+ Would it have been better to do it in Dask? Why?


For large datasets, it would be better to stay within Dask to calculate the moving average. Dask is designed to handle large datasets that don’t fit into memory by distributing computations across chunks and using parallel processing. 
However, if the dataset is relatively small and can fit comfortably into memory, converting to Pandas may make calculations faster and simpler, as Pandas operations are typically more optimized for in-memory data processing.



(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.